<img src='img/anaconda-logo.png' align='left' style="padding:10px">
<br>
*Copyright Continuum 2012-2016 All Rights Reserved.*

# Creating Conda Packages

Conda packages are the building blocks of working environments. 

They are the fundamental unit of organization and order in an otherwise disordered universe of working environments composed of many software library installs. They serve an organizational unit in a way very much analogous to that of python modules and python packages, as organizational units of collections of software.

In this lesson, we'll see how to create, install, and distribute your own conda packages, whether for use internally or in sharing with a broader, more open community.

An overview of creating and installing python packages serves to prepare for conda packages in two ways:
* a python package provides the basic ingredient needed to create a conda package
* the process of python package creation introduces concepts needed to understand conda packaging

## Table of Contents
* [Creating Conda Packages](#Creating-Conda-Packages)
	* [Before building a conda package](#Before-building-a-conda-package)
* [Building a Python Package](#Building-a-Python-Package)
	* [Creating a python module](#Creating-a-python-module)
	* [Creating a python package](#Creating-a-python-package)
* [Installing a Python Package](#Installing-a-Python-Package)
	* [2-Step Install](#2-Step-Install)
		* [Step 1: Create a `setup.py` file](#Step-1:-Create-a-setup.py-file)
		* [Step 2: Execute the setup.py install command](#Step-2:-Execute-the-setup.py-install-command)
	* [Install Paths](#Install-Paths)
	* [Uninstalling Python Packages](#Uninstalling-Python-Packages)
* [Building and Installing conda Packages](#Building-and-Installing-conda-Packages)
	* [Before you invoke conda](#Before-you-invoke-conda)
	* [Creating a Recipe from Scratch](#Creating-a-Recipe-from-Scratch)
	* [Creating a Recipe from Skeletons](#Creating-a-Recipe-from-Skeletons)
	* [Building a Conda Package from a Recipe](#Building-a-Conda-Package-from-a-Recipe)
	* [Install a conda package](#Install-a-conda-package)
* [Installing from Channels](#Installing-from-Channels)
	* [Default Channels](#Default-Channels)
	* [Uploading Packages](#Uploading-Packages)
	* [Uploading to Anaconda Cloud](#Uploading-to-Anaconda-Cloud)
* [Custom Channels](#Custom-Channels)
	* [Repository Channels](#Repository-Channels)
	* [Local Channels](#Local-Channels)
		* [Preparing the Local Chanel](#Preparing-the-Local-Chanel)
		* [Searching Local Channels](#Searching-Local-Channels)
		* [Installing from Local Channel](#Installing-from-Local-Channel)
	* [HTTP Serving Local Channels](#HTTP-Serving-Local-Channels)
		* [Prepared your conda config](#Prepared-your-conda-config)
		* [Start HTTP Server](#Start-HTTP-Server)
		* [Test HTTP Service](#Test-HTTP-Service)
	* [Exercise: Install from your HTTP Server](#Exercise:-Install-from-your-HTTP-Server)
	* [Exercise: Have others Install from your HTTP Server](#Exercise:-Have-others-Install-from-your-HTTP-Server)
* [Recap and Preview](#Recap-and-Preview)


## Before building a conda package

There are several things to keep in mind before building a conda package.


(1) A conda package is a file archive that contains ***executable*** programs.

* think of it as a container of a software library.

(2) Conda is ***not just for python***. A conda package may be composed of:

* purely interpreted programs, like Python or R
* purely compiled programs, like C++ or Fortran
* a mixture of interpreted and compiled programs.

(3) ***Building*** conda packages sometimes requires compiling

* This is an implication of point (1) and point (2)
* For our demonstration, we will make a conda package from pure Python: compiling in this case is not needed.
* If you wish to build a conda package for a software library which does contain source written in a compiled language (e.g. C, C++), the only change to the conda packaging process is that you must supply a build script capable of compiling your code. 

(4) ***Installing*** conda packages does NOT require compiling

* This is an implication of point (1)
* While creation of the conda package may require a compiler, installation of a conda package does not.
* Installing with conda means NOT having to install nor maintain your own set of compilers and build tools.
* Many conda packages have been created by Continuum Analytics and are available for free download and install using conda.

# Building a Python Package

## Creating a python module

Below is an example of a simple python module called `constants.py`.

```bash
print("Welcome to the constants.py module")

import math

E = 2.71828
PI = 3.14159

def compute_pi():
    return 4.0*math.atan(1.0)
    
def compute_e():
    sum = 0
    for n in range(10):
        sum += 1/math.factorial(n)
    return sum

def print_const(x):
    print("The value of your constant is ", x)
    return None

```

<img src='img/topics/Exercise.png' align='left'>
<div class='alert alert-warning' align="center">
<font size="+2">Create a python module</font>

<br/><br/>
<div style="padding-left: 10%" align="left">
Using the "New" menu on the Jupyter home tab, or your favorite text editor, create a new file `constants.py` within the `Conda` lesson directory, and paste the python code above into the file.
</div></div>

This module contains a print statement, one import, two "data attributes", `E` and `PI`, and three functions, `compute_pi()`, `compute_e()`, and `print_const()`.

This module is **executable** from the system shell.

```bash
cd Conda
python constants.py
```

This module is also **executable** from the python shell; it is not "compiled" before it is used.

```bash
cd Conda
python
```

```python
>>> exec(open("constants.py").read())
```

<img src='img/topics/Exercise.png' align='left'>
<div class='alert alert-warning' align="center">
<font size="+2">Execute the module as a script</font>

<br/><br/>
<div style="padding-left: 10%" align="left">
Instead of the python shell, execute the `constants.py` module using the IPython shell, using the `%run` magic followed by the file name.
</div></div>

From the python shell you can import this file as a python module, and thus access its attributes and call its functions:

```bash
cd Conda
python
```

```python
>>> import constants
>>> constants.PI
3.14
>>> constants.compute_pi()
3.1415926535897931
```

<img src='img/topics/Exercise.png' align='left'>
<div class='alert alert-warning' align="center">
<font size="+2">Import the module</font>

<br/><br/>
<div style="padding-left: 10%" align="left">
Using the IPython shell, launched either from the Jupyter home tab or your system shell, import the constants file as a module. Test the import by calling the `constants.compute_e()` method.
</div></div>

## Creating a python package

In python, building a large library of reusable code starts with organizing your code into modules, which are then further organized into a python package.

A python package is a collection of 

* one or more python modules (like `constants.py`)
* a build/install file called `setup.py`
* other optional and customary "meta-data files" like `README.txt`

These are organized into a particular directory structure. An example of a very simple python package is given below:

```bash
Desktop/
    my_package/
        constants.py
        setup.py
        README.txt
        LICENSE.txt
```

<img src='img/topics/Exercise.png' align='left'>
<div class='alert alert-warning' align="center">
<font size="+2">Create a python package</font>

<br/><br/>
<div style="padding-left: 10%" align="left">
Create the file tree shown above. Use the previously created constants.py. Create but otherwise leave all other files empty for now.
</div></div>

<img src='img/topics/Exercise.png' align='left'>
<div class='alert alert-warning' align="center">
<font size="+2">Create a `setup.py` file</font>

<br/><br/>
<div style="padding-left: 10%" align="left">
Edit the empty `setup.py` file and paste into it the code shown below.
</div></div>

```python
from setuptools import setup
setup( name='constants', 
       version='0.0.1', 
       py_modules=['constants',], 
     )
```

<img src='img/topics/Exercise.png' align='left'>
<div class='alert alert-warning' align="center">
<font size="+2">Add meta-data to your python package</font>

<br/><br/>
<div style="padding-left: 10%" align="left">
Edit the `README.txt` and `LICENSE.txt` files. For the `README`, add a brief description of `constants.py`. For the `LICENSE` file, add the following text within it:
</div></div>

> *Licensees may copy, distribute, display and perform the work and make derivative works and remixes based on it only if they give the author or licensor the credits (attribution) in the manner specified by these.*

# Installing a Python Package

The complete details of the process by which a python package is installed are beyond the scope of this present lesson. But the basics are simple: the contents of the `my_package` directory are in essense copied into a directory whose location will be known to any instance of the python interpreter you launch on your computer.

## 2-Step Install

<img src='img/topics/Advanced-Concept.png' align='left' style='padding:10x'>
<div class="alert alert-danger" align="center">
<font size="+1">The most common method for reproducibly and safely installing a Python package is outlined below:</font>
</div>

### Step 1: Create a `setup.py` file

```python
from setuptools import setup
setup( name='constants',
       version='0.0.1',
       py_modules=['constants',], 
     )
```

### Step 2: Execute the setup.py install command

```bash
python setup.py install
```

or

```bash
pip install .
```

...not yet!

For a software package that includes compiled source code, like a Python *extension package* or a C library, your build process would include an actual compilation step, possibly involving a Makefile (Linux), CMake file (Linux, Mac, Windows), or a VS Project file (Windows).

<img src='img/topics/Exercise.png' align='left'>
<div class='alert alert-warning' align="center">
<font size="+2">Make a more complete <tt>setup.py</tt></font>

<br/><br/>
<div style="padding-left: 10%" align="left">
Edit your `setup.py` file so that it reads as shown below.
</div></div>

Here is a more complete `setup.py` example:

```python
from setuptools import setup
setup(
    name='constants',
    version='0.0.1',
    license='CC BY-NC-SA 4.0',
    long_description=open('README.txt').read(),
    py_modules=['constants',],
    )
```

<img src='img/topics/Exercise.png' align='left'>
<div class='alert alert-warning' align="center">
<font size="+2">Update your <tt>LICENSE.txt</tt> file</font>

<br/><br/>
<div style="padding-left: 10%" align="left">
Search online for the license called "CC BY-NC-SA 4.0" to discover what it is if you have not heard of it. Replace all text within your `LICENSE.txt` with that of the "CC" license found online.
</div></div>

Review the following description of Creative Commons as well: https://en.wikipedia.org/wiki/Creative_Commons_license

<img src='img/topics/Exercise.png' align='left'>
<div class='alert alert-warning' align="center">
<font size="+2">Use <tt>setup.py</tt> to install the package</font>

<br/><br/>
<div style="padding-left: 10%" align="left">
Execute the `python setup.py install` command. Then test the install by moving to `Desktop`, launching a new python interpreter, and trying to `import constants`.
</div></div>

```bash
cd Desktop
python
```

```python
>>> import constants
Welcome to the constants.py module!
>>> constants.PI
3.14159
>>> constants.compute_pi()
3.1415926535897931
```

## Install Paths

How does the python `import` know where to find where the package was installed?

This is done through the `sys.path` directory list:

```python
>>> import sys
>>> sys.path
```

This `sys.path` is analogous to the system `PATH` variable. 

It is the list of directories on your file system wherein python will search for python modules or python packages when you try to `import` them.

When you installed your python package, its files will be located within one of the directories listed in `sys.path`. 

*Note: It is best to never try to copy files manually into any of the directories listed in `sys.path` as it is very easy to break something in the process, and it is very hard to reproduce the results even when successful.*

<img src='img/topics/Exercise.png' align='left'>
<div class='alert alert-warning' align="center">
<font size="+2">View path order in <tt>sys.path</tt></font>

<br/><br/>
<div style="padding-left: 10%" align="left">
Inspect the file path search order in `sys.path` by running the folling commands from the Python shell.
</div></div>

```python
>>> import sys
>>> sys.path
```

<img src='img/topics/Exercise.png' align='left'>
<div class='alert alert-warning' align="center">
<font size="+2">View python package install paths</tt></font>

<br/><br/>
<div style="padding-left: 10%" align="left">
Compare the `sys.path` file paths to those of the python package `site`, execute the following commands from within your Python interpreter.
</div></div>

```python
>>> import site
>>> site.USER_SITE
/Users/juser/.local/lib/python3.5/site-packages

>>> site.USER_BASE
/Users/juser/.local

>>> site.PREFIXES
/Users/juser/anaconda
```

## Uninstalling Python Packages

This is where things get painful. There is no `python setup.py uninstall`. You must find and manually delete all the installed files.

One helpful tool is to store the names of all the files at the time of install:

```
python setup.py install --record installed_files.txt
```

Contents of `installed_files.txt`
```
/Users/juser/anaconda/lib/python3.5/site-packages/constants-0.0.1-py3.5.egg
```

<img src='img/topics/Advanced-Concept.png' align='left' style='padding:10x'>
<div class="alert alert-danger" align="center">
<font size="+1">Python package left-overs</font>

<br/><br/>
<div style="padding-left: 10%" align="left">
It's even a bit more involved than that. Inspect the contents of the directory where the setup.py file lives, and you'll find many other intermediate file artifacts lieft-over from the build and install process.
</div></div>

<img src='img/topics/Advanced-Concept.png' align='left' style='padding:10x'>
<div class="alert alert-danger" align="center">
<font size="+1">The <tt>.egg</tt> and <tt>.whl</tt> file formats</font>

<br/><br/>
<div style="padding-left: 10%" align="left">
The <tt>.egg</tt> file was an early distribution format for Python packages. It’s an alternative to a source code distribution or an executable binary. Nowadays, outside the conda ecosystem, the "wheel" (<tt>.whl</tt>) format has supplanted "eggs"—both remain common though.  Both formats are cross-platform, and are at their heart <tt>.zip</tt> compressed containers that contain source code, and meta-data, and other resources.
</div></div>

<img src='img/topics/Exercise.png' align='left'>
<div class='alert alert-warning' align="center">
<font size="+2">Manually remove the install file</tt></font>

<br/><br/>
<div style="padding-left: 10%" align="left">
Uninstall the package by manually deleting the file listed in `installed_files.txt`. Test this "uninstall" by then launching a python shell and trying to import the package as `import constants`.
</div></div>

# Building and Installing conda Packages

Conda packages replace eggs and wheels.  They are also based on compressed containers (although using the smaller `.bz2` format), and contain more flexible and powerful metadata structure.  Best of all, conda packages work with the conda package and environment manager, and with Anaconda Repository; we recommend this format.

Conda itself does not create Python packages for you, but it does help you install, uninstall, and manage them. 

All conda asks in return, is that you "wrap" your packages inside container called a "conda package" so that conda can keep track of a wide variety of packages **and their interdependencies** in a consistent way.

The outline of the process to create a conda package is as follows:

1. write a software library
2. write a script that builds and/or installs this library
3. write a conda recipe
4. create a conda package from the conda recipe using conda-build

## Before you invoke conda

Regardless or the languages involved, whether compiled or interpreted, to build a conda package, your code must already be **executable** and **installable** without conda. 

By **"executable"** we mean that it must run without error if written in an interpreted language, and must build without error if written in a compiled language.

## Creating a Recipe from Scratch

Once your code is executable, you are ready to start creating a conda recipe.

A conda recipe is very much what its name implies: a list of raw ingredients and a list of steps by which those ingredients are processed and combined so as to generate something new: in this case, a conda package.

```bash
Desktop/
    my_recipe/
        meta.yaml
        build.sh
        bld.bat
```

The files within the directory `my_recipe` make up the conda recipe:

```bash
meta.yaml
build.sh
bld.bat
```

The files of the recipe serve the following purposes:

* one `meta.yaml` file, which defines build and run-time dependencies
* two `build scripts`, which compile source code (if needed) and install executable programs
* notice that there is one build script `build.sh` for Linux, and one `bld.bat` for Windows. 

Example contents of the recipe files are as follows:

* Contents of `meta.yaml`:

```yaml
package:
  name: constants
  version: "0.0.1"

source:
  path: ../my_package

requirements:
  build:
    - python
    - setuptools
  run:
    - python

test:
  imports:
    - constants

about:
  license: CC BY-NC-SA 4.0
  license_file: LICENSE.txt
```

* Contents of `build.sh`: 
	
```bash
python setup.py install
```

* Contents of `bld.bat`: 
	
```bash
"%PYTHON%" setup.py install
if errorlevel 1 exit 1
```

<img src='img/topics/Exercise.png' align='left'>
<div class='alert alert-warning' align="center">
<font size="+2">Create a Conda Recipe</tt></font>

<br/><br/>
<div style="padding-left: 10%" align="left">
Create the file tree shown above, and use a text editor to copy and paste the contents above into the files.
</div></div>

## Creating a Recipe from Skeletons

It is not always necessary to create a recipe from scratch.

There is a large collection of Python packages at the site [Python Package Index](pypi.python.org). You can ask conda to build a recipe based on a package hosted on `pypi`.

<img src='img/topics/Exercise.png' align='left'>
<div class='alert alert-warning' align="center">
<font size="+2">Skeleton Recipe</font>

<br/><br/>
<div style="padding-left: 10%" align="left">
Use the following command to create a conda recipe with the conda skeleton tool. This requires access to the website https://pypi.python.org/pypi
<pre>
conda skeleton pypi pyinstrument
</pre>
</div></div>

Confirm that you now have a directory with the following contents:

```bash
pyinstrument/
   meta.yaml
   build.sh
   bld.bat
```

Inspect the contents of the files created above with a text editor such as NotePad, TextEdit, or Emacs.

## Building a Conda Package from a Recipe

Now that you have a recipe, composed of the three files shown above, you can build the conda package from it. The process diagram might looks like this:

> (recipe + ingredients) → (conda-build) → (conda package)

The actual commands will be as follows:

* installing conda build, from your shell:

	```bash
	conda install conda-build
	```

* recall where your conda recipe was located:

	```bash
	Desktop
	    my_recipe
	        meta.yaml
	        build.sh
            bld.bat
	```

* using conda build to build from a recipe

    ```bash
    cd Desktop
    conda build my_recipe
    ```
* output from conda build is conda package, which takes the form of a compressed file archive with a name like `constants-0.0.1-py35_0.tar.bz2`

Likely output path will be:

* Linux or Mac-OSX:

	```bash
	 $HOME/anaconda/conda-bld/osx-64/`
	```

* Windows:

	```bash
	 %HOME%\anaconda\conda-bld\win-64\`
	```

<img src='img/topics/Exercise.png' align='left'>
<div class='alert alert-warning' align="center">
<font size="+2">Build a Conda Package</font>

<br/><br/>
<div style="padding-left: 10%" align="left">
Install `conda-build`, and use it to build both recipes created above: `my_recipe` and `pyinstruments`. Compare the outputs.
</div></div>

The output will be quite verbose, but should end with a message similar to the following block:

```
# If you want to upload this package to anaconda.org later, type:
#
# $ anaconda upload /Users/juser/anaconda/conda-bld/osx-64/constants-0.0.1-py35_0.tar.bz2
#
# To have conda build upload to anaconda.org automatically, use
# $ conda config --set anaconda_upload yes
```

<img src='img/topics/Exercise.png' align='left'>
<div class='alert alert-warning' align="center">
<font size="+2">Build using <tt>setup.py</tt> instead of <tt>build.sh</tt></font>

<br/><br/>
<div style="padding-left: 10%" align="left">
Remove the `build.sh` and `bld.bat` files from the recipe directory, and add the following block of code to the `meta.yaml`. Then try to rebuild the conda package from the resulting simplified recipe which contains only the updated `meta.yaml` file.
</div></div>

```python
    build:
        script: python setup.py install
```

## Install a conda package

As preparation, install `ipython` into the same environment in which you intend to install your new `constants` conda package, else you may get the system or `root` install of `ipython`

Before we install, let's make sure we remove to old "manually" installed version of the package:

```bash
/Users/juser/anaconda/lib/python3.5/site-packages/constants-0.0.1-py3.5.egg
```

Installing a locally created conda package is done as follows:

```bash
conda install --use-local constants
```

```
juser:~ ] $ conda install --use-local constants
Fetching package metadata .........
Solving package specifications: ..........
Using Anaconda Cloud api site https://api.anaconda.org

Package plan for installation in environment /Users/juser/anaconda:

The following NEW packages will be INSTALLED:

    constants: 0.0.1-py35_0 local

Proceed ([y]/n)?
```

<img src='img/topics/Exercise.png' align='left'>
<div class='alert alert-warning' align="center">
<font size="+2">Installing your conda package</font>

<br/><br/>
<div style="padding-left: 10%" align="left">
Create a new conda environment called `test1`. Then use `conda install --use-local constants` to install the local package.
</div></div>

```bash
conda create -n test1
source activate test1
conda install --use-local constants
```

Then test the exercise install in two ways:

* Launch the system shell, and run `conda list`, and see if `constants` is listed.
* Launch the IPython shell, and import the package with `import constants`.

<img src='img/topics/Exercise.png' align='left'>
<div class='alert alert-warning' align="center">
<font size="+2">Installing your package using the full path</font>

<br/><br/>
<div style="padding-left: 10%" align="left">
Create a new conda environment called `test2`. Then install the package by referencing the file path directly, as show below.
</div></div>

```bash
conda create -n test2 python=3.5 ipython
source activate test2
conda install /path/to/constants-0.0.1-py35_0.tar.bz2
```

In this case `/path/to/` is the location of the conda package output from running `conda-build`. Modify the actual package file name to match exactly what was output by `conda-build`.

<img src='img/topics/Exercise.png' align='left'>
<div class='alert alert-warning' align="center">
<font size="+2">Test the previous install by importing</font>

<br/><br/>
<div style="padding-left: 10%" align="left">
Install this package using the command above and then open a python interpreter and test the install by trying to import and use the package as follows.
</div></div>

```python
>>> import constants
>>> constants.PI
3.14159
>>> constants.compute_pi()
3.1415926535897931
```

<img src='img/topics/Exercise.png' align='left'>
<div class='alert alert-warning' align="center">
<font size="+2">Rebuild the package for python 2.7</font>

<br/><br/>
<div style="padding-left: 10%" align="left">
Create a new conda environment for Python 2.7 and activate it. Then use `conda build --python=2.7 constants` to rebuild the conda package. Once built, install it and test it as was done above with Python 3.
<hr/>
*Hint: the <tt>print()</tt> function in the <tt>constants.py</tt> file is compatible with Python 3 but behaves differently in Python 2. Make any necessary syntax changes or imports needed.*
</div></div>

# Installing from Channels

* Conda packages are usually installed from channels, not just from local builds.
* Channels are the paths (URLS, really) that conda searches for packages.
* The easiest way to use channels is to use a private (Anaconda Repository) or public (`anaconda.org`) repository with pre-existing channels.
* Custom channels can be constructed locally, and installed from using `file://` URLs, using just conda.

## Default Channels

* `anaconda.org` is the default repository from which conda installs packages
* `defaults` defines the default channels on `anaconda.org` which conda searches when asked to install a package.
* in the previous example we had to force conda to **not** look for `anaconda.org`
* we used the `--use-local` flag to install the local constants package. 

> For example, if want to install `numpy`, run `conda install numpy`. The numpy package file would be copied from from a channel called `default` that is hosted by Continuum Analytics at `repos.continuum.io`

## Uploading Packages

Recall the message that was seen at the end of a successful conda package build:

```
# If you want to upload this package to anaconda.org later, type:
#
# $ anaconda upload /Users/juser/anaconda/conda-bld/osx-64/constants-0.0.1-py35_0.tar.bz2
#
# To have conda build upload to anaconda.org automatically, use
# $ conda config --set anaconda_upload yes
```

## Uploading to Anaconda Cloud

This is very common way of sharing conda packages with the community of conda users. It first requires creating a account on `anaconda.org`.

* uploading to anaconda.org personal account
* installing from anaconda.org personal account
* how your distant collaborators can also install from anaconda.org

The steps are as follows:

* `conda install anaconda-client`
* `anaconda login`
* `conda config --set anaconda_upload yes`
* `anaconda upload $HOME/anaconda/conda-bld/osx-64/constants-0.0.1-py27_0.tar.bz2`

<img src='img/topics/Exercise.png' align='left'>
<div class='alert alert-warning' align="center">
<font size="+2">Install your package using anaconda.org</font>

<br/><br/>
<div style="padding-left: 10%" align="left">
Install your recently uploaded package from anaconda.org using the below command.
</div></div>

```bash
conda install constants --channel <YOUR-ANACONDA-USERNAME>
```

# Custom Channels

## Repository Channels

You can use channels that are not the defaults on repositories like Anaconda Cloud or a private instance of Anaconda Repository.

To do so, you must add the channel to your conda figuration as follows:

```bash
conda config --add channels http://anaconda.org/<USERNAME>
```

## Local Channels

You can create custom local channels as well. 

While it is best to use Anaconda Cloud or Anaconda Repository, this example is useful in understanding how channels work.

*This demonstration was developed and tested using `conda 4.1.11`*

### Preparing the Local Chanel

Preparing the local channel requires the following steps:
* find or build at least one conda package
* create a file tree for your channel:
    * top level dir is the channel name
    * sub-dirs for platform specific packages
* place your conda packages (files.tar.bz2) into the path
* run `conda index` on the platform sub-dirs

```bash
conda build my_recipe
mkdir /opt/my_channel/osx-64/
cd ~/anaconda/conda-bld/osx-64/
cp constants-0.0.1-py35_0.tar.bz2 /opt/my_channel/osx-64/
conda index /opt/my_channel/osx-64/
```

You can use `conda convert` to add packages for other platforms to your local channel:

```bash
mkdir /opt/my_channel/win-64/
cd ~/anaconda/conda-bld/osx-64/
conda convert constants-0.0.1-py35_0.tar.bz2 -p win-64
cp win-64/constants-0.0.1-py35_0.tar.bz2 /opt/my_channel/win-64/
conda index /opt/my_channel/win-64/
```

### Searching Local Channels

After preparing your channel, test it with `conda seach`:

```bash
conda search -c file://opt/my_channel constants --override-channels
```

### Installing from Local Channel

Once prepared and tested, you are ready to install from your channel:
* create a new environment
* activate that environment
* install a package from your local channel with `conda install`
    * use `-c file://full_path_to_channel`
    * also use the `--override-channels` flag

```bash
cd ~/
conda create -n test-local-channel python=3.5 ipython
source activate test-local-channel
conda install -c file://opt/my_channel constants --override-channels
```

## HTTP Serving Local Channels

You can even serve your local channels over HTTP:

*Note: this is leading us to the capability that Anaconda Cloud and Anaconda Repository provide.*

### Prepared your conda config

Update your `~/.condarc` by adding the IP address as shown:

```bash
channels:
- http://0.0.0.0:8000
- defaults
```

### Start HTTP Server

Use python to start an HTTP server:

Start the HTTP server from the directory that contains your indexed `osx-64` and `win-64` subdirs, so that they appear at the top level of the file tree served out:

```bash
cd ~/Desktop/my_channel
python -m http.server
```

### Test HTTP Service

Open a web browser to verify that you can see the directory contents, but crowsing to the following URL:

```
http://0.0.0.0:8000
```

TO install from this HTTP served channel, use `conda install` and specifiy the `<URL>:<PORT>` as the channel:

```bash
conda create -n test python=3.5
source activate test
conda install -c http://0.0.0.0:8000 constants
```

## Exercise: Install from your HTTP Server

Using `http://127.0.0.1:8000` in your `~/.condarc` and then install with `conda install -c http://local.host constants`. Again, use a web browser and try to load `http://127.0.0.1` to see if you get a file listing of the contents of `my_channel`

```
#~/.condarc
channels:
- http://127.0.0.1:8000
- defaults
```

## Exercise: Have others Install from your HTTP Server

Find you assigned IP address on the local network. For example, on Linux of Mac, in a terminal, run the `ifconfig` command, and look for the value of `inet` in the output block labeled `en0`. Give that IP address to your classmate to your left, and ask them to run the commands above, but replace `0.0.0.0` or `127.0.0.1` with this new address. Does it work?

```
#~/.condarc
channels:
- http://10.0.32.184:8000
- defaults
```

```bash
conda create -n test-http python=3.5
source activate test-http
conda install -c http://10.0.32.184:8000 constants --override-channels
```

# Recap and Preview

In this lesson we saw....

* Building a Python module
* Building a Python package
* Building a Conda package
* Installing a local conda package
* Installing packages from channels
* Uploading a custom conda package to anaconda.org
* Creating a custom local package channel
* Serving a local channel with a HTTP service

---
*Copyright Continuum 2012-2016 All Rights Reserved.*